In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics.pairwise import manhattan_distances

In [19]:
class LogicDataset:

  def __init__(self, variablesCount, flavour):
    fileName = f'004_logic_{variablesCount}_{flavour}.h5'
    self.model = pd.read_hdf(fileName, "model")
    self.dataset = pd.read_hdf(fileName, "data")
    self.X = self.dataset.drop("result", axis=1)
    self.y = self.dataset["result"].copy()

data_2 = LogicDataset(4, 2)
data_3 = LogicDataset(4, 3)

In [6]:
def fitSGDClassfier(X, y, loss, max_iter=1000, n_iter_no_change=5):
    sgd = SGDClassifier(
        loss=loss, learning_rate="optimal", 
        max_iter=max_iter, n_iter_no_change=n_iter_no_change, 
        shuffle=True, random_state=42, n_jobs=-1)
    sgd.fit(X, y)
    print(f'n_iter: {sgd.n_iter_}')
    return sgd

def print_preditions(X, y, y_pred):
  print(f'{"X":15}| {"y":3}| {"pred":3}')
  for i in range(0, len(data_2.X)):
    print(f'{str(X.iloc[i].to_list()):15}| {str(y[i]):3}| {str(y_pred[i]):3}')

In [4]:
data_2.model

,1_1_1_1,1_1_1_2,1_1_2_1,1_1_2_2,1_2_1_1,1_2_1_2,1_2_2_1,1_2_2_2,2_1_1_1,2_1_1_2,2_1_2_1,2_1_2_2,2_2_1_1,2_2_1_2,2_2_2_1,2_2_2_2
0,0,1,1,0,0,1,0,1,0,0,1,1,1,1,1,1


## Classical

In [15]:
clf_log = LogisticRegression(random_state=42, solver='lbfgs').fit(data_2.X, data_2.y)
clf_log_y = clf_log.predict(data_2.X)
clf_log_py = clf_log.predict_proba(data_2.X)
print_preditions(data_2.X, data_2.y, [f'{clf_log_y[i]} ({clf_log_py[i][1]})' for i in range(0, len(clf_log_y))] )

X              | y  | pred
[0, 0, 0, 0]   | 1  | 1 (0.832444619195386)
[0, 0, 0, 1]   | 1  | 1 (0.7453054763976655)
[0, 0, 1, 0]   | 1  | 1 (0.7453054763976655)
[0, 0, 1, 1]   | 1  | 1 (0.6328367251262011)
[0, 1, 0, 0]   | 1  | 1 (0.7453054763976655)
[0, 1, 0, 1]   | 1  | 1 (0.6328367251262011)
[0, 1, 1, 0]   | 0  | 1 (0.6328367251262011)
[0, 1, 1, 1]   | 0  | 1 (0.5037705116288859)
[1, 0, 0, 0]   | 1  | 1 (0.7453054763976655)
[1, 0, 0, 1]   | 0  | 1 (0.6328367251262011)
[1, 0, 1, 0]   | 1  | 1 (0.6328367251262011)
[1, 0, 1, 1]   | 0  | 1 (0.5037705116288859)
[1, 1, 0, 0]   | 0  | 1 (0.6328367251262011)
[1, 1, 0, 1]   | 1  | 1 (0.5037705116288859)
[1, 1, 1, 0]   | 1  | 1 (0.5037705116288859)
[1, 1, 1, 1]   | 0  | 0 (0.3741998293635806)


In [8]:
clf_sgd = fitSGDClassfier(data_2.X, data_2.y, loss='hinge')
clf_sgd_y = clf_sgd.predict(data_2.X)
print_preditions(data_2.X, data_2.y, clf_sgd_y)

n_iter: 10
X              | y  | pred
[0, 0, 0, 0]   | 1  | 1  
[0, 0, 0, 1]   | 1  | 1  
[0, 0, 1, 0]   | 1  | 1  
[0, 0, 1, 1]   | 1  | 1  
[0, 1, 0, 0]   | 1  | 1  
[0, 1, 0, 1]   | 1  | 0  
[0, 1, 1, 0]   | 0  | 0  
[0, 1, 1, 1]   | 0  | 0  
[1, 0, 0, 0]   | 1  | 1  
[1, 0, 0, 1]   | 0  | 1  
[1, 0, 1, 0]   | 1  | 1  
[1, 0, 1, 1]   | 0  | 0  
[1, 1, 0, 0]   | 0  | 0  
[1, 1, 0, 1]   | 1  | 0  
[1, 1, 1, 0]   | 1  | 0  
[1, 1, 1, 1]   | 0  | 0  


In [16]:
clf_svc = SVC(kernel='rbf').fit(data_2.X, data_2.y)
clf_svc_y = clf_svc.predict(data_2.X)
manhattan_distances([data_2.y], [clf_svc_y])[0][0]

1.0

In [17]:
clf_svc2 = SVC(kernel='poly', degree=2).fit(data_2.X, data_2.y)
clf_svc2_y = clf_svc2.predict(data_2.X)
manhattan_distances([data_2.y], [clf_svc2_y])[0][0]

1.0

In [18]:
clf_svc3 = SVC(kernel='poly', degree=3).fit(data_2.X, data_2.y)
clf_svc3_y = clf_svc3.predict(data_2.X)
manhattan_distances([data_2.y], [clf_svc3_y])[0][0]

0.0

## Fuzzy

In [29]:
data_3_y = [1.0 if y == 1 else 0.0 for y in data_3.y]

In [30]:
clf_svc_3 = SVC(kernel='rbf').fit(data_3.X, data_3_y)
clf_svc_3_y = clf_svc_3.predict(data_3.X)
manhattan_distances([data_3_y], [clf_svc_3_y])[0][0]

10.0

In [31]:
for i in range(2, 10):
  clf_svc_i = SVC(kernel='poly', degree=i).fit(data_3.X, data_3_y)
  clf_svc_i_y = clf_svc_i.predict(data_3.X)
  print(f'{i}: manhattan distance = {manhattan_distances([data_3_y], [clf_svc_i_y])[0][0]}')

2: manhattan distance = 10.0
3: manhattan distance = 9.0
4: manhattan distance = 5.0
5: manhattan distance = 3.0
6: manhattan distance = 1.0
7: manhattan distance = 1.0
8: manhattan distance = 1.0
9: manhattan distance = 1.0
